# Merge "skin" and "skull" masks into single mask

Zahra's new data from April 19 2023 consists of two sets of mask images: "skin_masks" and "skull_masks". The masks in these folders are binary labeled for "skin" and "skull" respectively. 

Hence, I'm going to merge these images into a single categorical mask with the following labeling scheme:

| Semantic label | Numeric label |
| -------------- | ------------- |
| "background"   |       0       |
| "skull"        |       1       |
| "skin"         |       2       |

## Merging process

Define function to return all images in a directory (and subdirectories)

In [1]:
import os

image_types = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")


def list_images(basePath, contains=None):
    # return the set of files that are valid
    return list_files(basePath, validExts=image_types, contains=contains)


def list_files(basePath, validExts=None, contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if validExts is None or ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename)
                yield imagePath

Define function to merge skin and skull masks into single mask

In [2]:
import numpy as np

def merge_boolean_masks(mask1:np.ndarray, mask2:np.ndarray, label1:int=1, label2:int=2):
    """
    Merge two boolean labeled numpy array image masks into a single array image
    mask that is categorically labeled from the input arrays.

    Args:
        mask1 (numpy.ndarray): First boolean labeled image mask.
        mask2 (numpy.ndarray): Second boolean labeled image mask.
        label1 (int, optional): Label value for pixels in the merged mask
            corresponding to True values in `mask1`. Defaults to 1.
        label2 (int, optional): Label value for pixels in the merged mask
            corresponding to True values in `mask2`. Defaults to 2.

    Returns:
        numpy.ndarray: Merged categorically labeled image mask.

    Raises:
        ValueError: If `mask1` and `mask2` do not have the same shape
            or if they are both not boolean
    """
    # check if masks are the same size
    if mask1.shape != mask2.shape:
        raise ValueError("Input masks must have the same shape.")
    # check if masks are boolean labeled
    if mask1.dtype != bool or mask2.dtype != bool:
        raise ValueError("Input masks must be boolean labeled.")

    # create a new array for the merged mask
    merged_mask = np.zeros_like(mask1).astype(int)

    # set the pixel values in the merged mask based on the input masks
    if mask1.any():
        merged_mask[mask1 == True] = label1
    if mask2.any():
        merged_mask[mask2 == True] = label2

    return merged_mask

Get filenames of all skin and skull masks from their respective directories

In [3]:
# Directories containing skin an skull masks
data_dir = "../data/raw/OCT_scans_new_20230419"
skull_dir = f"{data_dir}/skull_mask"
skin_dir = f"{data_dir}/skin_mask"

# Get all filenames for skin and skull masks
skull_fnames = list(list_images(skull_dir))
skin_fnames = list(list_images(skin_dir))
skin_basenames = [os.path.basename(fname) for fname in skin_fnames]


Merge all images that are common

In [4]:
from PIL import Image
import tifffile

# Define output directory for merged images
merged_dir = f"{data_dir}/merged_masks"
if not os.path.isdir(merged_dir):
    print(f"Creating merged directory: {merged_dir}")
    os.makedirs(merged_dir)

# Iterate through all the skull/skin images and merge them
for ind, skull_fname in enumerate(skull_fnames):
    # Find match between skin and skull mask
    skull_basename = os.path.basename(skull_fname)
    if skull_basename not in skin_basenames:
        print(f"No skull-skin images match for {skull_basename}")
        skull = np.array(Image.open(skull_fname))
        skin = np.zeros_like(skull).astype(bool)
    else:
        skull = np.array(Image.open(skull_fname))
        skin_fname = skin_fnames[skin_basenames.index(skull_basename)]
        skin = np.array(Image.open(skin_fname))

    # Merge images and save (using uint8 to make smaller images)
    merged = merge_boolean_masks(skull, skin).astype(np.uint8)
    merged_fname = os.path.join(merged_dir, skull_basename)
    assert "tif" in merged_fname
    tifffile.imwrite(merged_fname, merged, compression="packbits")